In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.environ.get('PATH_CUSTOM_MODULES'))

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path_data = os.environ.get('PATH_DATA_DASHBOARD')
path_aug_exm = os.path.join(path_data, 'augment_sample')

### Refine the scenario guide data

In [11]:
scen_guide = pd.read_csv(os.path.join(path_data, 'scenario_guide.csv'))

scen_guide.color_schema = scen_guide.color_schema.str.upper()
scen_guide.rename(columns={'scenario': 'Skenario',
                            'color_schema': 'Warna',
                            'aug_resize': 'Resize',
                            'aug_rescale': 'Rescale',
                            'aug_horizontal_flip': 'Flip Horizontal',
                            'aug_vertical_flip': 'Flip Vertical',
                            'aug_rotation': 'Rotasi',
                            'aug_brightness': 'Brightness',
                            'aug_contrast_clahe': 'CLAHE',},
                    inplace=True)
scen_guide.replace(to_replace=[1, 0], value=[True, False], inplace=True)
scen_guide.replace(to_replace={'Skenario': True}, value=1, inplace=True)

scen_guide.style.hide()

C:\Users\Bugi\AppData\Local\Temp\ipykernel_19728\821726218.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  scen_guide.replace(to_replace=[1, 0], value=[True, False], inplace=True)
C:\Users\Bugi\AppData\Local\Temp\ipykernel_19728\821726218.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  scen_guide.replace(to_replace={'Skenario': True}, value=1, inplace=True)


Skenario,Warna,Resize,Rescale,Flip Horizontal,Flip Vertical,Rotasi,Brightness,CLAHE
1,RGB,True,True,False,False,False,False,False
2,RGB,True,True,True,True,True,True,False
3,RGB,True,True,True,True,True,True,True


In [12]:
scen_guide.to_csv(os.path.join(path_data, 'rfn_scenario_guide.csv'), index=False)

### Manage the augment example

In [37]:
aug_files = pd.DataFrame({'file_name': os.listdir(path_aug_exm)})

aug_files.drop(aug_files[aug_files.file_name.str.contains('paper')].index, inplace=True)
aug_files['scenario'] = aug_files.file_name.str.split('_').str[1].str.split(' ').str[0]
aug_files['dataset'] = aug_files.file_name.str.split(' ').str[1]
aug_files['augmentation'] = aug_files.file_name.str.split(' ').str[2].str.split('.').str[0]

aug_files.head(5)

,file_name,scenario,dataset,augmentation
8,scenario_1 g1020 no.png,1,g1020,no
9,scenario_1 papila no.png,1,papila,no
10,scenario_1 refuge no.png,1,refuge,no
11,scenario_1 rimone no.png,1,rimone,no
12,scenario_2 g1020 bright.png,2,g1020,bright


In [38]:
aug_files.to_csv(os.path.join(path_data, 'example_aug_imgs.csv'), index=False)

In [46]:
list(aug_files.loc[(aug_files.scenario == '2')
            & (aug_files.augmentation == 'bright'), 'file_name'].values)

['scenario_2 g1020 bright.png',
 'scenario_2 papila bright.png',
 'scenario_2 refuge bright.png',
 'scenario_2 rimone bright.png']

### ---

### Analysis

In [3]:
data_stored = r'./../../../data/evaluation_result/'
data_final = pd.read_csv(os.path.join(data_stored,'merge_result_final.csv'))
data_raw = pd.read_csv(os.path.join(data_stored,'merge_result.csv'))

In [51]:
data_final

,scenario,model,dataset,auc,f1_score,accuracy,precision,sensitivity,loss,true_positive,true_negative,false_positive,false_negative
0,scenario_1,mobilenet_v2,refuge,0.918615,0.963669,0.933333,0.945633,0.982407,0.203795,212.2,11.8,12.2,3.8
1,scenario_2,mobilenet_v2,refuge,0.917940,0.947418,0.906667,0.960952,0.934259,0.250824,201.8,15.8,8.2,14.2
2,scenario_3,mobilenet_v2,rimone,0.910473,0.888535,0.855670,0.885714,0.891374,0.464054,55.8,27.2,7.2,6.8
3,scenario_2,mobilenet_v2,rimone,0.908892,0.881203,0.837113,0.832386,0.936102,0.441865,58.6,22.6,11.8,4.0
4,scenario_1,mobilenet_v2,rimone,0.891411,0.852201,0.806186,0.839009,0.865815,0.441153,54.2,24.0,10.4,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,scenario_2,mobilenet_v2,g1020,0.489268,0.750653,0.625490,0.711634,0.794199,0.719550,115.0,12.6,46.6,29.8
68,scenario_3,efficientnet_v2m,g1020,0.488724,0.830275,0.709804,0.709804,1.000000,0.604689,144.8,0.0,59.2,0.0
69,scenario_1,mobilenet_v2,g1020,0.483521,0.795620,0.670588,0.710870,0.903315,0.711355,130.8,6.0,53.2,14.0
70,scenario_1,efficientnet_v2m,g1020,0.475572,0.830275,0.709804,0.709804,1.000000,0.603954,144.8,0.0,59.2,0.0


In [8]:
len(data_final)

72

In [13]:
len(data_final.loc[data_final.scenario == 'scenario_3'])

24

In [32]:
list(data_final.model.unique())

['mobilenet_v2',
 'efficientnet_v2s',
 'mobilenet_v3small',
 'efficientnet_v2l',
 'efficientnet_v2m',
 'mobilenet_v3large']

In [42]:
temp_df = {}
for name in list(data_final.model.unique()):
    temp = data_final.loc[data_final.model == name]
    temp.sort_values(by=['scenario', 'auc', 'f1_score', 'accuracy'], ascending=False, inplace=True)
    temp.reset_index(drop=True, inplace=True)

    temp_df[name] = temp.iloc[[0,4,8],:6]

C:\Users\Bugi\AppData\Local\Temp\ipykernel_12504\3554909041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.sort_values(by=['scenario', 'auc', 'f1_score', 'accuracy'], ascending=False, inplace=True)
C:\Users\Bugi\AppData\Local\Temp\ipykernel_12504\3554909041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.sort_values(by=['scenario', 'auc', 'f1_score', 'accuracy'], ascending=False, inplace=True)
C:\Users\Bugi\AppData\Local\Temp\ipykernel_12504\3554909041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [43]:
main_merge = pd.concat(temp_df.values(), ignore_index=True)

In [44]:
main_merge

,scenario,model,dataset,auc,f1_score,accuracy
0,scenario_3,mobilenet_v2,rimone,0.910473,0.888535,0.855670
1,scenario_2,mobilenet_v2,refuge,0.917940,0.947418,0.906667
2,scenario_1,mobilenet_v2,refuge,0.918615,0.963669,0.933333
3,scenario_3,efficientnet_v2s,refuge,0.828183,0.948109,0.901667
4,scenario_2,efficientnet_v2s,refuge,0.748823,0.947276,0.900000
5,scenario_1,efficientnet_v2s,refuge,0.704610,0.947784,0.900833
6,scenario_3,mobilenet_v3small,rimone,0.731954,0.718750,0.591753
7,scenario_2,mobilenet_v3small,refuge,0.610359,0.947368,0.900000
8,scenario_1,mobilenet_v3small,refuge,0.597242,0.947368,0.900000
9,scenario_3,efficientnet_v2l,rimone,0.715976,0.778231,0.663918


In [45]:
main_merge.dataset.value_counts()

dataset
rimone    11
refuge     7
Name: count, dtype: int64

In [50]:
main_merge[['model', 'dataset','scenario']].groupby(by=['model','dataset']).count().rename(columns={'scenario':'count'})

count
model             dataset       
efficientnet_v2l  rimone       3
efficientnet_v2m  rimone       3
efficientnet_v2s  refuge       3
mobilenet_v2      refuge       2
                  rimone       1
mobilenet_v3large rimone       3
mobilenet_v3small refuge       2
                  rimone       1